<a href="https://colab.research.google.com/github/jbust97/DeepWhatsapp/blob/main/Analisis_Datos_regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from functools import reduce
import statsmodels.api as sm
from scipy import stats


# Configuraciones

In [11]:
## Setting ####
# Configuraciones del dataset a usar
# Fecha de inicio de conteo de casos acumulados
CUM_DATE_0 = "2023-07-29"
# Fecha de fin de conteo de casos acumulados
CUM_DATE_F = "2024-01-10"
# Clasificación de casos: SOSPECHOSO, PROBABLE, CONFIRMADO, TOTAL (S+P+C)
CLASSIFICATION_0 =  "CONFIRMADO"
# Nivel: National, Departamento, Eje
LEVEL_0  =  None #"National"
# Nombre de la localidad
NAME_0  = "ALTO PARAGUAY"
# Enfermedad: DENGUE, CHIKUNGUNYA
DISEASE_0 =  "CHIKUNGUNYA"


# argumentos para rollmean media movil
ROLL_MEAN = 3 # periodo
MEAN_ALIGN =  "right" # Alineación de medias calculadas
FILL_MEAN = None # Valor de relleno (Cómo alternativa se puede implementar un metodo de interpolación)

# Semanas de entrenamiento
TRAINING_WINDOW_DAYS = 28
FORECASTING_WINDOW_DAYS = 21
FORECASTING_WINDOW_WEEKS = 5

In [12]:
# Read the CSV file
data_csv = pd.read_csv(
    "case_data_2019_2023_upto16sep_23_weekly_v20231103.csv",
    header=0,
    sep=",",
    decimal=".",
    encoding="UTF-8"
)

# Preparacion de datos

In [13]:
# Filter rows based on conditions
df_0 = data_csv[(data_csv['disease'] != "DENGUE,CHIKUNGUNYA") & (data_csv['disease'] != "ARBOVIROSIS")]

if DISEASE_0:
  df_0 = df_0[df_0['disease'] == DISEASE_0]
if LEVEL_0:
  df_0 = df_0[df_0['level'] == LEVEL_0]
if NAME_0:
  df_0 = df_0[df_0['name'] == NAME_0]
if CLASSIFICATION_0:
  df_0 = df_0[df_0['classification'] == CLASSIFICATION_0]

# Update the 'i_cases' column in the DataFrame 'df_0' where the value is 0 with a random exponential value between 1e-9 and 1.
df_0.loc[df_0['i_cases'] == 0, 'i_cases'] = np.exp(np.random.uniform(np.log(1e-9), np.log(1)))

# Group by specified columns and apply mutations
df_0 = df_0.groupby(['name', 'level', 'disease', 'classification'],group_keys=False).apply(lambda group: (
    group.assign(csum=(group['i_cases'].cumsum() + np.exp(np.random.uniform(np.log(1e-9), np.log(1)))),
                 id_proy=group['name'] + '-' + group['disease'] + '-' + group['classification'])
)).reset_index(drop=True)
# create columns of time
df_0['date'] = pd.to_datetime(df_0['date'])
df_0['t'] = (df_0['date'] - df_0['date'].min() + (1 * np.timedelta64(1, 'D')))  / np.timedelta64(1, 'D')


In [14]:
def adj_r_squared(y_true, y_pred, n, p):
    r_squared = stats.linregress(y_true, y_pred).rvalue ** 2
    adj_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))
    return adj_r_squared

Thera are two functions that can be used to calculated the predicted, lower and upper values. They differ in how they calculate lower and upper values.

In [15]:
#In this version of the predict function, the confidence interval is calculated using the training set. This looks counterintuitive to me. We can discuss about this later.

def predict(model, new_data, x, y, confidence_level=0.90):
    # Predicting the mean response
    predicted_values = model.predict(new_data)
    y_pred = model.predict(x)
    squared_errors = (y - y_pred) ** 2
    # Calculate mean squared error
    mse = np.mean(squared_errors)
    sep = np.sqrt(mse / (y_pred.shape[0] - x.shape[1] - 1))
    # TODO: Verificar que se calcule asi el intervalo de confianza
    # Organizing the results into a DataFrame
    return pd.DataFrame({'fit': predicted_values, 'upr': predicted_values + confidence_level*sep, 'lwr': predicted_values - confidence_level*sep})



In [16]:
#In this version of the predict function, the confidence interval is calculated using only the testing set. I think this is the right way to do this.
def predict2(model,x,y,confidence_level=0.90):
    y_pred = model.predict(x)
    squared_errors = (y - y_pred) ** 2
    # Calculate mean squared error
    mse = np.mean(squared_errors)
    sep = np.sqrt(mse / (y_pred.shape[0] - x.shape[1] - 1))
    # TODO: Verificar que se calcule asi el intervalo de confianza

    n = x.shape[0]
    p = model.coef_.shape[0]-1
    AIC = n * np.log(np.mean((y - y_pred) ** 2)) + 2 * p
    AICc = AIC + (2 * p * (p + 1)) / (n - p - 1)
    BIC = n * np.log(np.mean((y - y_pred) ** 2)) + p * np.log(n)



    # Organizing the results into a DataFrame
    return pd.DataFrame(
        {
         'observed': y,
         'fit': y_pred,
         'upr': y_pred + confidence_level*sep,
         'lwr': y_pred - confidence_level*sep,
         #+ 1 for the intercept
         'rank': np.sum(model.coef_ != 0) + 1  ,
         'Df.res':  n - x.shape[1] - 1,
         'Adj_R_sq': adj_r_squared(y,y_pred,n,p+1),
         'AIC': AIC,
         'AICc': AICc,
         'BIC': BIC
         })


In [ ]:
# Para cada nivel de datos que tenemos
for level in df_0['id_proy'].unique():
  # dataframe base
  df_1 = df_0[df_0['id_proy'] == level]
  for i in range(5, df_1.shape[0] + 1):
    training_dataset = df_1.iloc[i-5:i]

    # Linear model 1) lm.e.tc: I(log(csum)) ~ t
    t = training_dataset['t']
    x = t.values.reshape(-1, 1)
    log_x = np.log(t.values).reshape(-1, 1)

    y = np.log(training_dataset['csum'])

    lm_e_tc = LinearRegression().fit(x, y)

    # Linear model 2) lm.se.tc: I(log(csum)) ~ log(t)
    lm_se_tc = LinearRegression().fit(log_x, y)


    # Linear model 3) lm.seco.tc: I(log(csum)) ~ log(t) + t
    lm_seco_tc = LinearRegression().fit(np.column_stack((log_x, x)), y)


    results = pd.DataFrame({
      'Model': ['log(TC)=log(a)+c t', 'log(TC)=log(a)+b log(t)', 'log(TC)=log(a)+b log(t)+c t'],
      'Method': ['Exponential', 'Sub-exponential', 'Sub-exponential-decay'],
      'log_a': [lm_se_tc.intercept_, lm_se_tc.intercept_, lm_seco_tc.intercept_],
      'b': [lm_se_tc.coef_[0],
            lm_se_tc.coef_[0],
            lm_seco_tc.coef_[0]],
      'c': [lm_se_tc.coef_[1] if hasattr(lm_e_tc, 'coef_') and len(lm_se_tc.coef_) > 1 else 0,
            0,
            lm_seco_tc.coef_[1] if hasattr(lm_seco_tc, 'coef_') and len(lm_seco_tc.coef_) > 1 else 0],
      'Fit.criteria': [np.sum((lm_se_tc.predict(x) - y) ** 2),
                      np.sum((lm_se_tc.predict(log_x) - y) ** 2),
                      np.sum((lm_seco_tc.predict(np.column_stack((log_x, x))) - y) ** 2)],
      'id': np.full(3, i)
    })
    ## Evaluation of models

    new = (t.max(skipna=True) + np.arange(0, FORECASTING_WINDOW_WEEKS * 7, 7))
    new_x = new.reshape(-1, 1)
    new_log_x = np.log(new).reshape(-1, 1)

    y = df_1[df_1['t'].isin(new)]['csum']


    # Linear model 1) lm.e.tc
    #data = predict(lm_e_tc, new_x, x, y)
    data = predict2(lm_e_tc, new_x, y)
    new = pd.DataFrame({'t': new})
    new['Method']= np.full(FORECASTING_WINDOW_WEEKS, 'Exponential')
    data['Method'] = new['Method']
    data['t'] = new['t']

    exponential = pd.merge(results, data, on='Method',how='right')

    # Linear model 2) lm.se.tc
    #data = predict(lm_se_tc, new_log_x, log_x, y)
    data = predict2(lm_se_tc, new_log_x, y)
    new['Method']= np.full(FORECASTING_WINDOW_WEEKS, 'Sub-exponential')
    data['t'] = new['t']
    data['Method'] = new['Method']



    sub_exponential = pd.merge(results, data, on='Method',how='right')

    # Linear model 3) lm.seco.tc

    #data = predict(lm_seco_tc, np.column_stack((new_log_x, new_x)), np.column_stack((log_x, x)), y)
    data = predict2(lm_seco_tc, np.column_stack((new_log_x, new_x)), y)
    new['Method']= np.full(FORECASTING_WINDOW_WEEKS, 'Sub-exponential-decay')
    data['t'] = new['t']
    data['Method'] = new['Method']


    sub_exponential_decay = pd.merge(results, data, on='Method',how='right')

    result = pd.concat([exponential, sub_exponential, sub_exponential_decay], ignore_index=True)
    result['Relative.likelihood'] =  np.exp((np.min(result['AICc']) - result['AICc'])/ 2)
    print(result.to_markdown())
    input()







|    | Model                       | Method                |       log_a |          b |           c |   Fit.criteria |   id |   observed |     fit |     upr |      lwr |   rank |   Df.res |   Adj_R_sq |     AIC |    AICc |     BIC |   t |   Relative.likelihood |
|---:|:----------------------------|:----------------------|------------:|-----------:|------------:|---------------:|-----:|-----------:|--------:|--------:|---------:|-------:|---------:|-----------:|--------:|--------:|--------:|----:|----------------------:|
|  0 | log(TC)=log(a)+c t          | Exponential           |  -0.13605   |   0.682688 |   0         |    553.833     |    5 |    13.0008 | 2.59456 | 11.0344 | -5.84525 |      2 |        3 |   0.959396 | 27.8763 | 27.8763 | 27.8763 |  57 |              1        |
|  1 | nan                         | nan                   | nan         | nan        | nan         |    nan         |  nan |    14.0008 | 3.19212 | 11.6319 | -5.24768 |      2 |        3 |   0.959396 | 27.8763 